In [55]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma

In [50]:
from dotenv import load_dotenv

load_dotenv()

True

In [51]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [8]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [9]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [53]:
descriptions_list = books["tagged_description"].tolist()
with open("tagged_description.txt", "w", encoding="utf-8") as f:
    for description in descriptions_list:
        f.write(f"{description}\n")

print("File 'tagged_description.txt' created successfully.")

File 'tagged_description.txt' created successfully.


In [52]:
raw_documents = TextLoader("tagged_description.txt", encoding='utf-8').load()
text_splitter = CharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=0,
    separator="\n"
)

documents = text_splitter.split_documents(raw_documents)
print("Successfully loaded and split the documents.")

Created a chunk of size 5800, which is longer than the specified 5000


Successfully loaded and split the documents.


In [37]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [54]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

db_books = Chroma.from_documents(
    documents,
    embedding=embeddings
)


In [38]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)

[Document(id='550d3990-2b08-431f-8afb-a3563b9b1b3d', metadata={'source': 'tagged_description.txt'}, page_content="9780786712342 A profile based on the subject's personal archives describes such topics as Doyle's education at Stonyhurst and Edinburgh University, his Southsea medical practice, his two marriages, his forays into politics and spiritualism, his friendships with such figures as H. G. Wells and King Edward VII, and his literary ambitions. Reprint.\n9780786712946 The victim of a Fourth of July gang rape, single mother Teena Maguire and her daughter become the target of harassment and violence on the part of the assailants after Teena identifies the perpetrators for the Niagara Falls Police Department. 75,000 first printing.\n9780786714827 The victim of a Fourth of July gang rape, single mother Teena Maguire and her daughter become the target of harassment and violence on the part of the assailants after Teena identifies the perpetrators for the Niagara Falls Police Department.

In [39]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3739,9780786712342,0786712341,The Life of Sir Arthur Conan Doyle,John Dickson Carr,Biography & Autobiography,http://books.google.com/books/content?id=QrU5P...,A profile based on the subject's personal arch...,2003.0,4.02,320.0,146.0,The Life of Sir Arthur Conan Doyle,9780786712342 A profile based on the subject's...


In [40]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [41]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
27,9780006545866,0006545866,The White Album,Joan Didion,American essays,http://books.google.com/books/content?id=qauOP...,This collection of essays recounts what took p...,1993.0,4.17,224.0,91.0,The White Album,9780006545866 This collection of essays recoun...
86,9780060000141,0060000147,Poppy's Return,Avi,Juvenile Fiction,http://books.google.com/books/content?id=XbcMJ...,"There's trouble at Gray House, the girlhood ho...",2006.0,3.99,256.0,1086.0,Poppy's Return,"9780060000141 There's trouble at Gray House, t..."
99,9780060094935,0060094931,The Bromeliad Trilogy,Terry Pratchett,Juvenile Fiction,http://books.google.com/books/content?id=IsFpv...,In a world whose seasons are defined by Christ...,2003.0,4.06,502.0,5681.0,"The Bromeliad Trilogy: Truckers, Diggers, and ...",9780060094935 In a world whose seasons are def...
106,9780060256753,0060256753,"Lafcadio, the Lion Who Shot Back",Shel Silverstein,Juvenile Fiction,http://books.google.com/books/content?id=lmeJR...,"""You don't have to shoot me,"" says the young l...",1963.0,4.15,112.0,5191.0,"Lafcadio, the Lion Who Shot Back","9780060256753 ""You don't have to shoot me,"" sa..."
386,9780061159176,0061159174,The Known World,Edward P. Jones,Fiction,http://books.google.com/books/content?id=NE-vs...,One of the most acclaimed novels in recent mem...,2006.0,3.83,388.0,28962.0,The Known World,9780061159176 One of the most acclaimed novels...
394,9780061312113,0061312118,The Blue and Brown Books,Ludwig Wittgenstein,Philosophy,http://books.google.com/books/content?id=ffiZu...,"These works, as the sub-title makes clear, are...",1965.0,4.13,208.0,1970.0,The Blue and Brown Books,"9780061312113 These works, as the sub-title ma..."
413,9780064405850,0064405850,Strawberry Girl 60th Anniversary Edition,Lois Lenski,Juvenile Fiction,http://books.google.com/books/content?id=AQXM2...,"The land was theirs, but so were its hardships...",1995.0,3.86,208.0,10655.0,Strawberry Girl 60th Anniversary Edition,"9780064405850 The land was theirs, but so were..."
435,9780064462341,006446234X,Pioneer Girl,William Anderson,Juvenile Nonfiction,http://books.google.com/books/content?id=Sj4UD...,The pioneer spirit lives on... Readers around ...,2000.0,4.15,32.0,414.0,Pioneer Girl: The Story of Laura Ingalls Wilder,9780064462341 The pioneer spirit lives on... R...
440,9780066238500,0066238501,The Chronicles of Narnia (adult),C. S. Lewis,Fiction,http://books.google.com/books/content?id=3VGkK...,"Journeys to the end of the world, fantastic cr...",2001.0,4.26,767.0,425445.0,The Chronicles of Narnia (adult),9780066238500 Journeys to the end of the world...
544,9780130235015,0130235016,Things Fall Apart,Chinua Achebe;Sandra Widner,Igbo (African people),NaN,"[This book is] a simple story of a ""strong man...",2000.0,3.64,113.0,40.0,Things Fall Apart: An Adapted Classic,9780130235015 [This book is] a simple story of...
